In [ ]:
import pandas as pd
import mysql.connector
import os

# Connect to MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='PASSWORD@629',
    database='inventory'
)
cursor = conn.cursor()

# Path to your data folder
folder_path = r'C:/Users/User/Desktop/Data Analytics/All projects/Vendor performance analysis project/Dataset/data/'

# Function to map pandas dtype to MySQL type
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

# Loop through all CSV files in folder
for csv_file in os.listdir(folder_path):
    if csv_file.lower().endswith('.csv'):
        table_name = os.path.splitext(csv_file)[0]  # table name = file name without extension
        file_path = os.path.join(folder_path, csv_file)
        
        # Read CSV
        df = pd.read_csv(file_path)
        df = df.where(pd.notnull(df), None)
        df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

        # Create table
        columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
        cursor.execute(f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})')

        # Insert rows
        for _, row in df.iterrows():
            values = tuple(None if pd.isna(x) else x for x in row)
            placeholders = ', '.join(['%s'] * len(row))
            sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({placeholders})"
            cursor.execute(sql, values)

        conn.commit()
        print(f"✅ Loaded {csv_file} into table `{table_name}`")

# Close connection
conn.close()


✅ Loaded begin_inventory.csv into table `begin_inventory`
✅ Loaded end_inventory.csv into table `end_inventory`
